## Adding weather features to the dataset

In [ ]:
import dask.dataframe as dd
import pandas as pd
from dask.diagnostics import ProgressBar
# Load your taxi data
OUTPUT_DIR = '/d/hpc/projects/FRI/bigdata/students/in7357/out'
taxi_df = dd.read_parquet(f"{OUTPUT_DIR}/optimized_parquet", engine="pyarrow")


In [18]:
WEATHER_DATA_PATH = "/d/hpc/projects/FRI/bigdata/students/in7357/weather_data.csv"

In [19]:
# Add column for hourly time (floored datetime)
taxi_df["pickup_datetime"] = dd.to_datetime(taxi_df["tpep_pickup_datetime"])
taxi_df["pickup_hour"] = taxi_df["pickup_datetime"].dt.floor("H")

# Load your new weather data
weather_df = dd.read_csv(
    WEATHER_DATA_PATH,
    parse_dates=["time"]
)

# Rename 'time' to 'pickup_hour' so we can join on it
weather_df = weather_df.rename(columns={"time": "pickup_hour"})

# Optionally clean column names (optional but good style)
weather_df = weather_df.rename(columns=lambda x: x.strip().lower().replace(" ", "_").replace("(", "").replace(")", "").replace("°", "deg"))

# Merge on pickup_hour (left join to keep all taxi rows)
augmented_df = dd.merge(
    taxi_df,
    weather_df,
    on="pickup_hour",
    how="left"
)

# Save to Parquet (or any format you want)

augmented_df.to_parquet(
    f"{OUTPUT_DIR}/augmented_with_weather_2020_2024",
    write_index=False
)

/d/hpc/home/in7357/.conda/envs/big_data311/lib/python3.11/site-packages/dask/dataframe/dask_expr/_accessor.py:113: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)
/d/hpc/home/in7357/.conda/envs/big_data311/lib/python3.11/site-packages/dask/dataframe/dask_expr/_accessor.py:113: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)


In [20]:
# Show the first few rows of the merged dataframe
augmented_df.head()


/d/hpc/home/in7357/.conda/envs/big_data311/lib/python3.11/site-packages/dask/dataframe/dask_expr/_accessor.py:113: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)
/d/hpc/home/in7357/.conda/envs/big_data311/lib/python3.11/site-packages/dask/dataframe/dask_expr/_accessor.py:113: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)
/d/hpc/home/in7357/.conda/envs/big_data311/lib/python3.11/site-packages/dask/dataframe/dask_expr/_accessor.py:113: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  out = getattr(getattr(obj, accessor, obj), attr)(*args, **kwargs)


,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,...,cloudcover_%,cloudcover_low_%,cloudcover_mid_%,cloudcover_high_%,windspeed_10m_km/h,winddirection_10m_deg,snowfall_cm,cloud_cover_%,surface_pressure_hpa,wind_speed_10m_km/h
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238.0,239.0,1.0,...,50,33,73,0,13.4,211,0.0,50,997.5,13.4
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239.0,238.0,1.0,...,50,33,73,0,13.4,211,0.0,50,997.5,13.4
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238.0,238.0,1.0,...,50,33,73,0,13.4,211,0.0,50,997.5,13.4
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238.0,151.0,1.0,...,50,33,73,0,13.4,211,0.0,50,997.5,13.4
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193.0,193.0,2.0,...,50,33,73,0,13.4,211,0.0,50,997.5,13.4


## Making new columns from school locations data

In [21]:
import zipfile
import geopandas as gpd
import os

zip_path = "/d/hpc/projects/FRI/bigdata/students/in7357/taxi_zones.zip"  
extract_dir = "/d/hpc/projects/FRI/bigdata/students/in7357/taxi_zones_shapefile"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

shp_file = [f for f in os.listdir(extract_dir) if f.endswith(".shp")][0]
gdf = gpd.read_file(os.path.join(extract_dir, shp_file)).to_crs("EPSG:4326")

# Compute centroids
gdf["latitude"] = gdf.centroid.y
gdf["longitude"] = gdf.centroid.x
zone_coords = gdf[["LocationID", "latitude", "longitude"]].copy()


/tmp/ipykernel_2048172/1347718353.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["latitude"] = gdf.centroid.y
/tmp/ipykernel_2048172/1347718353.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["longitude"] = gdf.centroid.x


Should not be an issue as it is a relativly small area.

### Map coordinates to pickup locations and dropoff locations


In [22]:
import pandas as pd
import dask.dataframe as dd

pickup_zones = zone_coords.rename(columns={
    "LocationID": "pulocationid",
    "latitude": "pickup_latitude",
    "longitude": "pickup_longitude"
})
dropoff_zones = zone_coords.rename(columns={
    "LocationID": "dolocationid",
    "latitude": "dropoff_latitude",
    "longitude": "dropoff_longitude"
})

taxi_df = dd.read_parquet(f"{OUTPUT_DIR}/optimized_parquet")
taxi_df['pulocationid'] = taxi_df['pulocationid'].astype('int32')
taxi_df['dolocationid'] = taxi_df['dolocationid'].astype('int32')
# Merge pickup and dropoff coordinates
taxi_df = taxi_df.merge(pickup_zones, on="pulocationid", how="left")
taxi_df = taxi_df.merge(dropoff_zones, on="dolocationid", how="left")

In [23]:
taxi_df.head()

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,year,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,...,0.0,0.3,11.27,2.5,None,2020,40.791705,-73.973049,40.783961,-73.978632
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,...,0.0,0.3,12.30,2.5,None,2020,40.783961,-73.978632,40.791705,-73.973049
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,...,0.0,0.3,10.80,2.5,None,2020,40.791705,-73.973049,40.791705,-73.973049
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,...,0.0,0.3,8.16,0.0,None,2020,40.791705,-73.973049,40.797962,-73.968168
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,...,0.0,0.3,4.80,0.0,None,2020,40.760314,-73.941997,40.760314,-73.941997


### Load school locations data

In [24]:
schools_df = pd.read_csv("/d/hpc/projects/FRI/bigdata/students/in7357/school_locations.csv")

def extract_coords(location_str):
    try:
        coordinates = location_str.split('\n')[-1].strip('(').strip(')').split(',')
        lat, long =  map(float, coordinates)
        return pd.Series([lat, long])
    except Exception:
        return pd.Series([None, None])
    

# def extract_coords(location_str):
#     import re
#     match = re.search(r'\\(([-\\d.]+), ([-\\d.]+)\\)', str(location_str))
#     return pd.Series((float(match[1]), float(match[2]))) if match else pd.Series((None, None))

schools_df[['school_lat', 'school_lon']] = schools_df['Location 1'].apply(extract_coords)
# schools_df = schools_df.dropna(subset=['school_lat', 'school_lon'])

school_lat = schools_df['school_lat'].values
school_lon = schools_df['school_lon'].values
print(len(school_lat))

1836


In [25]:
schools_df.head(2)

,FISCAL_YEAR,ATS SYSTEM CODE,LOCATION_CODE,LOCATION_NAME,BEDS NUMBER,MANAGED_BY_NAME,LOCATION_TYPE_DESCRIPTION,LOCATION_CATEGORY_DESCRIPTION,GRADES_TEXT,GRADES_FINAL_TEXT,...,FIELD_SUPPORT_CENTER_NAME,FIELD_SUPPORT_CENTER_LEADER_NAME,SCHOOL_SUPPORT_TEAM_NAME,SCHOOL_SUPPORT_TEAM_LEADER_NAME,HIGHSCHOOL_NETWORK_LOCATION_CODE,HIGHSCHOOL_NETWORK_NAME,HIGHSCHOOL_NETWORK_SUPERINTENDENT,Location 1,school_lat,school_lon
0,2016,01M015,M015,P.S. 015 Roberto Clemente,310100010015,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",...,Field Support Center - Manhattan,"CHU, YUET",School Support Team 3- Manhattan,NaN,NaN,NaN,NaN,"333 EAST 4 STREET\nMANHATTAN, NY 10009\n(40.72...",40.722075,-73.978747
1,2016,01M019,M019,P.S. 019 Asher Levy,310100010019,DOE,General Academic,Elementary,"PK,0K,01,02,03,04,05,SE","PK,0K,01,02,03,04,05",...,Field Support Center - Manhattan,"CHU, YUET",School Support Team 3- Manhattan,NaN,NaN,NaN,NaN,"185 1 AVENUE\nMANHATTAN, NY 10003\n(40.730009,...",40.730009,-73.984496


### Define Haversine distatance function and feature logic

In [26]:
import numpy as np
from numba import jit

@jit(nopython=True)
def haversine_numba(lat1, lon1, lat2, lon2):
    """
    Numba-optimized haversine distance calculation for single point vs array
    Returns distances in km
    """
    R = 6371.0
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

def enrich_with_school_features(df, school_lat, school_lon):
    if df.empty or len(school_lat) == 0:
        return df.assign(
            pickup_nearest_school_distance_km=np.nan,
            pickup_near_school=0,
            pickup_school_count_500m=0,
            dropoff_near_school=0
        )
    
    # Convert to numpy arrays once
    lat_pick = df['pickup_latitude'].values
    lon_pick = df['pickup_longitude'].values
    lat_drop = df['dropoff_latitude'].values
    lon_drop = df['dropoff_longitude'].values
    
    # Pre-allocate result arrays
    n = len(df)
    pickup_min_dist = np.full(n, np.inf)
    pickup_count_500m = np.zeros(n, dtype=np.int32)
    dropoff_min_dist = np.full(n, np.inf)
    
    # Process schools in batches to control memory usage
    batch_size = 100  # Adjust based on available memory
    n_schools = len(school_lat)
    
    for i in range(0, n_schools, batch_size):
        batch_end = min(i + batch_size, n_schools)
        batch_lat = school_lat[i:batch_end]
        batch_lon = school_lon[i:batch_end]
        
        # Process pickup locations
        for j in range(len(batch_lat)):
            dists = haversine_numba(lat_pick, lon_pick, batch_lat[j], batch_lon[j])
            # Use np.fmin to handle NaN values properly 
            pickup_min_dist = np.fmin(pickup_min_dist, dists)
            pickup_min_dist = np.where(pickup_min_dist == np.inf, np.nan, pickup_min_dist)
            # Replace any NaN values with original distance
            # pickup_min_dist = np.nan_to_num(pickup_min_dist, nan=np.inf)
            
            pickup_count_500m += (dists <= 0.5)
            
            # Process dropoff locations
            dists = haversine_numba(lat_drop, lon_drop, batch_lat[j], batch_lon[j])
            dropoff_min_dist = np.fmin(dropoff_min_dist, dists)
            # cast inf to NaN
            dropoff_min_dist = np.where(dropoff_min_dist == np.inf, np.nan, dropoff_min_dist)
            # dropoff_min_dist = np.nan_to_num(dropoff_min_dist, nan=np.inf)
            # dropoff_min_dist = np.minimum(np.nanmin(dropoff_min_dist), np.nanmin(dists))
    
    return df.assign(
        pickup_nearest_school_distance_km=pickup_min_dist,
        pickup_near_school=(pickup_min_dist <= 0.2).astype(float),
        pickup_school_count_500m=pickup_count_500m,
        dropoff_nearest_school_distance_km=dropoff_min_dist,
        dropoff_near_school=(dropoff_min_dist <= 0.2).astype(float)
    )




In [27]:
sample_df = taxi_df.sample(frac=0.0001, random_state=42).compute()

In [28]:
result = enrich_with_school_features(
    sample_df,
    school_lat,
    school_lon)


In [ ]:
# works on a small sample
result.head()

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,...,year,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,pickup_nearest_school_distance_km,pickup_near_school,pickup_school_count_500m,dropoff_nearest_school_distance_km,dropoff_near_school
4332439,1.0,2020-01-22 20:10:13,2020-01-22 20:28:03,1.0,2.20,1.0,N,100,113,1.0,...,2020,40.753513,-73.988787,40.732579,-73.994305,0.468796,0.0,1,0.364154,0.0
3594116,2.0,2020-01-18 23:54:57,2020-01-19 00:04:55,6.0,1.96,1.0,N,264,264,1.0,...,2020,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,0.0
554958,1.0,2020-01-04 11:10:51,2020-01-04 11:18:37,3.0,1.30,1.0,N,161,237,2.0,...,2020,40.758028,-73.977698,40.768615,-73.965635,0.487379,0.0,1,0.546982,0.0
4400510,2.0,2020-01-23 08:05:22,2020-01-23 08:17:23,5.0,1.73,1.0,N,141,162,1.0,...,2020,40.766948,-73.959635,40.756688,-73.972356,0.146181,1.0,7,0.518991,0.0
3076650,2.0,2020-01-16 17:09:03,2020-01-16 17:12:50,1.0,0.93,1.0,N,263,236,1.0,...,2020,40.778766,-73.951010,40.780436,-73.957012,0.326397,0.0,5,0.125957,1.0


### Calculate features

In [31]:
import pyarrow as pa

schema = {
    "vendorid": pa.float64(),
    "tpep_pickup_datetime": pa.timestamp('us'),
    "tpep_dropoff_datetime": pa.timestamp('us'),
    "passenger_count": pa.float64(),
    "trip_distance": pa.float64(),
    "ratecodeid": pa.float64(),
    "store_and_fwd_flag": pa.string(),
    "pulocationid": pa.int32(),
    "dolocationid": pa.int32(),
    "payment_type": pa.float64(),
    "fare_amount": pa.float64(),
    "extra": pa.float64(),
    "mta_tax": pa.float64(),
    "tip_amount": pa.float64(),
    "tolls_amount": pa.float64(),
    "improvement_surcharge": pa.float64(),
    "total_amount": pa.float64(),
    "congestion_surcharge": pa.float64(),
    "airport_fee": pa.string(),
    "year": pa.dictionary(pa.int8(), pa.int32()),
    "pickup_latitude": pa.float64(),
    "pickup_longitude": pa.float64(),
    "dropoff_latitude": pa.float64(),
    "dropoff_longitude": pa.float64(),
    "pickup_nearest_school_distance_km": pa.float64(),
    "pickup_near_school": pa.float64(),
    "pickup_school_count_500m": pa.float64(),
    "dropoff_near_school": pa.float64(),
    "dropoff_nearest_school_distance_km": pa.float64(),
}

# Convert the schema to a PyArrow schema
arrow_schema = pa.schema(schema)


In [32]:
len(taxi_df)

32857944

In [33]:
meta = taxi_df._meta.copy()
meta['pickup_nearest_school_distance_km'] = 'f8'
meta['pickup_near_school'] = 'f8'  # <- now float, for nans
meta['pickup_school_count_500m'] = 'f8'
meta['dropoff_nearest_school_distance_km'] = 'f8'  # <- now float, for nans
meta['dropoff_near_school'] = 'f8'  # <- now float

# Assuming school_lat and school_lon are predefined NumPy arrays
taxi_enriched = taxi_df.map_partitions(
    enrich_with_school_features,
    school_lat,  # Pass school data as arguments
    school_lon,
    meta=meta
)



# output_path = f"{OUTPUT_DIR}/augmented_with_schools_only"
# taxi_enriched.to_parquet(output_path, write_index=False,
#                         schema =arrow_schema)

output_path = f"{OUTPUT_DIR}/augmented_with_schools_optimized"



In [34]:
taxi_enriched = taxi_enriched.repartition(partition_size="200MB")

In [ ]:
# This takes a lot of time, since we can't use parrallelism on SLING
with ProgressBar():
    taxi_enriched.to_parquet(
        output_path,
        write_index=False,
        schema=arrow_schema
    )

[############                            ] | 31% Completed | 30m 43sss